In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import numpy as np

In [3]:
# list of available datasets
print(tfds.list_builders()[:5])

['abstract_reasoning', 'aeslc', 'aflw2k3d', 'ai2_arc', 'amazon_us_reviews']


In [4]:
imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised= True)

In [5]:
info.description

'Large Movie Review Dataset.\nThis is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.'

In [6]:
imdb.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [7]:
train_data , test_data = imdb['train'] , imdb['test']

training_sentences = []
test_sentences = []

training_labels = []
test_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    test_sentences.append(s.numpy().decode('utf8'))
    test_labels.append(l.numpy())

In [8]:
training_labels_final = np.array(training_labels)
test_labels_final = np.array(test_labels)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences


vocab_size = 10000
embeding_size = 16
max_lenght =120
trunc_type= 'post'
oov_token ='<OOV>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen= max_lenght, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_lenght, truncating=trunc_type)

In [10]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()]) 

In [11]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

In [12]:
print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [13]:
len(padded[3])

120

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embeding_size, input_length=max_lenght),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')    
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epoch = 10
model.fit(padded, training_labels_final, epochs=num_epoch, validation_data=(testing_padded, test_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 6ms/step - loss: 0.5023 - accuracy: 0.7276 - val_loss: 0.3795 - val_accuracy: 0.8302
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2343 - accuracy: 0.9114 - val_loss: 0.4402 - val_accuracy: 0.8063
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0810 - accuracy: 0.9796 - val_loss: 0.5226 - val_accuracy: 0.8066
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0179 - accuracy: 0.9982 - val_loss: 0.6105 - val_accuracy: 0.8076
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0045 - accuracy: 0.9998 - val_loss: 0.6858 - val_accuracy: 0.8088
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.7461 - val_accuracy: 0.8076
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 8.1514e-04 - accuracy: 1.0000 - val_loss: 0.7950 - val_accuracy: 0.8091
Ep

In [16]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)
# shape: (vocab_size, embedding_dim)

(10000, 16)


In [17]:
weights[0]

array([-0.04648294,  0.04688442, -0.0237039 ,  0.00412041, -0.03263653,
        0.00210918, -0.01246622,  0.05807818,  0.03839532, -0.03560852,
       -0.01723221,  0.02857719,  0.04197186, -0.01259835,  0.0133344 ,
       -0.01624592], dtype=float32)

In [19]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1,vocab_size):
    
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    
out_v.close()
out_m.close()